In [4]:
import pandas as pd
import numpy as np
#import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import distributions
from torch.nn.parameter import Parameter
import torch.utils.data as data_utils
#from core import * 
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 7.0)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Please download data 
https://archive.ics.uci.edu/ml/machine-learning-databases/00373/drug_consumption.data


and put it in the same directory than this notebook

In [7]:
def read_drug(shuffle=False):
    data = pd.read_csv(
    "drug_consumption.data",
        engine='python',
        na_values="?", header=None)
    if shuffle:
        data = data.sample(frac=1).reset_index(drop=True)
    nTrain = data.shape[0] - data.shape[0]//10
    for c in [0]+list(range(13,24))+list(range(25,32)):
        del data[c]
    data.rename(columns={24:'Target'},inplace=True)
    data['Target'] = 1*(data['Target'] != 'CL0')
    data['NotWhite'] = 1*(data[5] != -0.31685)
    data = pd.get_dummies(data, columns=['NotWhite'])
    to_protect = data['NotWhite_0'].values
    cols = data.columns.tolist()
    n = int(cols.index('NotWhite_0'))
    cols = cols[n:n+2] + cols[:n] + cols[n+2:]  
    data = data[cols]
    #for n in ['credit_amount', 'age', 'installment_as_income_perc']:
    #    data[n] = (data[n]-data[n].mean())/data[n].std()
    data.fillna(0,inplace=True)
    return data, to_protect #data.iloc[:nTrain, :], to_protect[:nTrain], data.iloc[nTrain:, :], to_protect[nTrain:]

data, to_protect = read_drug(shuffle=True)
data.head()    

,NotWhite_0,NotWhite_1,1,2,3,4,5,6,7,8,9,10,11,12,Target
0,1,0,-0.07854,-0.48246,0.45468,-0.57009,-0.31685,0.22393,-1.37639,0.58331,-2.78793,-0.00665,0.88113,1.22470,1
1,0,1,-0.95197,0.48246,-0.61113,-0.57009,-1.10702,1.37297,-2.03972,0.88309,-0.01729,-1.13788,-0.21712,0.40148,0
2,1,0,-0.07854,0.48246,-0.61113,-0.57009,-0.31685,0.52135,1.93886,1.43533,-0.30172,0.93949,-0.71126,1.22470,0
3,1,0,1.09449,-0.48246,1.16365,0.96082,-0.31685,-1.32828,1.28610,-0.17779,0.59042,1.81175,-2.55524,-1.18084,0
4,1,0,1.09449,-0.48246,1.16365,0.96082,-0.31685,-0.92104,-0.30033,-1.11902,0.59042,1.63088,-0.21712,-0.52593,0


In [8]:
Y = data.Target
X = data
A = X.NotWhite_0
X = X.drop('NotWhite_0', 1)
X = X.drop('NotWhite_1', 1)
X = X.drop('Target', 1)
#X = pd.DataFrame(list(X.items()),columns=feature_names)
X.to_csv('drug_X.csv', sep='\t')
A.to_csv('drug_A.csv', sep='\t')
Y.to_csv('drug_Y.csv', sep='\t')

/Users/omid/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  if __name__ == '__main__':
/Users/omid/.local/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  # Remove the CWD from sys.path while we load stuff.


In [3]:
# Hyper Parameters 
input_size = encoded_data.shape[1]-1
num_classes = 2
num_epochs = 50
batch_size = 8
batchRenyi = 128.
learning_rate = 3e-4
lambda_renyi = 4. * batchRenyi/batch_size

class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        out = self.linear(x)
        return out

class NetRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NetRegression, self).__init__()
        size = 40
        self.first = nn.Linear(input_size, size)
        self.fc = nn.Linear(size, size-20)
        self.fc2 = nn.Linear(size-20, size)
        self.last = nn.Linear(size, num_classes)
        self.d = nn.Dropout(0.5)
         
    
    def forward(self, x):
        out = F.selu( self.first(x) )
        #out = self.d(out)
        #out = F.selu( self.fc(out) )
        #out = F.selu( self.fc2(out) )    
        out = self.last(out)
        return out
    
    
cfg_factory=namedtuple('Config', 'model loader batch_size num_epochs lambda_renyi batchRenyi learning_rate input_size num_classes' )
config = cfg_factory(NetRegression, read_drug, batch_size, num_epochs, lambda_renyi, batchRenyi, learning_rate, input_size, num_classes)

In [4]:
results_1 = pd.DataFrame([go(config,deleteSensitiveInfo=True, reg=False) for _ in range(10)])
results_1.describe()

/Users/j.mary/venv/lib/python3.7/site-packages/numpy/lib/function_base.py:3937: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above


,accuracy,balanced_acc,deo_0.30,deo_0.34,deo_0.38,deo_0.42,deo_0.46,deo_0.50,deo_0.54,deo_0.58,...,di_0.42,di_0.46,di_0.50,di_0.54,di_0.58,di_0.62,di_0.66,di_0.70,loss,renyi
count,10.000000,10.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,10.000000,10.000000,10.000000,10.000000,8.000000,2.000000,1.000000,0.0,10.000000,10.000000
mean,0.792553,0.825946,0.283584,0.239150,0.207045,0.216080,0.208905,0.150618,0.084788,0.064372,...,inf,inf,inf,inf,inf,inf,inf,NaN,0.455301,0.141573
std,0.027006,0.030105,0.169937,0.153157,0.140393,0.107656,0.123750,0.159954,0.125974,0.153813,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.048282,0.066621
min,0.739362,0.770174,0.068182,0.030303,0.013158,0.092105,0.071429,0.026316,0.026316,0.000000,...,0.372093,0.217054,0.125749,0.418182,0.139394,inf,inf,NaN,0.402053,0.079263
25%,0.777926,0.804426,0.194444,0.131579,0.090909,0.135135,0.108108,0.054054,0.032258,0.000000,...,0.854661,0.492314,0.533149,inf,inf,inf,NaN,NaN,0.417288,0.098582
50%,0.803191,0.830473,0.215054,0.204545,0.258065,0.181818,0.171053,0.064516,0.045455,0.022727,...,1.255439,1.324101,inf,inf,inf,inf,NaN,NaN,0.439910,0.117331
75%,0.812500,0.849304,0.328947,0.354839,0.295455,0.290323,0.258065,0.138889,0.055556,0.027027,...,inf,inf,inf,inf,inf,inf,NaN,NaN,0.494735,0.174114
max,0.819149,0.861582,0.571429,0.516129,0.419355,0.386364,0.409091,0.454545,0.418919,0.472973,...,inf,inf,inf,inf,inf,inf,inf,NaN,0.531324,0.289592


In [5]:
results_2 = pd.DataFrame([go(config,deleteSensitiveInfo=True, floor=True) for _ in range(10)])
results_2.describe()

,accuracy,balanced_acc,deo_0.30,deo_0.34,deo_0.38,deo_0.42,deo_0.46,deo_0.50,deo_0.54,deo_0.58,...,di_0.42,di_0.46,di_0.50,di_0.54,di_0.58,di_0.62,di_0.66,di_0.70,loss,renyi
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000,0.0,0.0,10.000000,10.000000
mean,0.782979,0.755131,0.250750,0.226085,0.217538,0.133022,0.123219,0.093758,0.058553,0.026559,...,inf,inf,inf,inf,inf,inf,NaN,NaN,0.460797,0.120522
std,0.017872,0.048296,0.276647,0.221973,0.229169,0.081131,0.096214,0.061114,0.047744,0.026179,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.027985,0.042735
min,0.760638,0.685140,0.025000,0.025000,0.010753,0.013393,0.032258,0.000000,0.000000,0.000000,...,0.395349,0.397661,0.217054,0.093023,0.000000,0.000000,NaN,NaN,0.416774,0.048133
25%,0.767287,0.711768,0.084005,0.086197,0.106967,0.072043,0.049988,0.051619,0.013158,0.000000,...,0.632663,0.535948,0.362504,0.304144,0.241446,0.000000,NaN,NaN,0.445968,0.099984
50%,0.779255,0.769449,0.096501,0.164187,0.143105,0.133333,0.089247,0.088542,0.062261,0.027047,...,0.789474,0.789981,0.618167,0.481306,inf,inf,NaN,NaN,0.464895,0.114045
75%,0.796543,0.787602,0.442302,0.255376,0.235887,0.203560,0.182950,0.122636,0.089063,0.046121,...,1.214553,1.153666,1.162435,inf,inf,inf,NaN,NaN,0.479781,0.144450
max,0.808511,0.823781,0.741935,0.743590,0.794872,0.230769,0.282051,0.200000,0.142857,0.064516,...,inf,inf,inf,inf,inf,inf,NaN,NaN,0.505065,0.204060


In [6]:
results_3 = pd.DataFrame([go(config,deleteSensitiveInfo=True) for _ in range(10)])
results_3.describe()

,accuracy,balanced_acc,deo_0.30,deo_0.34,deo_0.38,deo_0.42,deo_0.46,deo_0.50,deo_0.54,deo_0.58,...,di_0.42,di_0.46,di_0.50,di_0.54,di_0.58,di_0.62,di_0.66,di_0.70,loss,renyi
count,10.000000,10.000000,9.000000,9.000000,9.000000,9.0,9.0,9.0,9.0,9.0,...,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.000000,10.000000
mean,0.784574,0.801341,0.045423,0.005393,0.002469,0.0,0.0,0.0,0.0,0.0,...,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.525905,0.166957
std,0.047128,0.062738,0.054956,0.010750,0.007407,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.057880,0.084299
min,0.734043,0.724286,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.453022,0.023011
25%,0.740691,0.763069,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.477395,0.113950
50%,0.771277,0.781170,0.045455,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.531971,0.163655
75%,0.824468,0.849813,0.052632,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.562583,0.211019
max,0.851064,0.916667,0.155556,0.026316,0.022222,0.0,0.0,0.0,0.0,0.0,...,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.620161,0.291514


In [7]:
results_4 = pd.DataFrame([go(config,reg=False) for _ in range(10)])
results_4.describe()

,accuracy,balanced_acc,deo_0.30,deo_0.34,deo_0.38,deo_0.42,deo_0.46,deo_0.50,deo_0.54,deo_0.58,...,di_0.42,di_0.46,di_0.50,di_0.54,di_0.58,di_0.62,di_0.66,di_0.70,loss,renyi
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,9.000000,4.000000,1.000000,0.0,10.000000,10.000000
mean,0.780851,0.794257,0.264527,0.299550,0.238469,0.154964,0.107328,0.070201,0.045419,0.018863,...,inf,inf,inf,inf,inf,inf,inf,NaN,0.465658,0.126374
std,0.036835,0.050856,0.195361,0.193660,0.107928,0.064566,0.048857,0.040770,0.043929,0.022157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.049700,0.058683
min,0.718085,0.706486,0.027907,0.041860,0.102941,0.044118,0.051282,0.000000,0.000000,0.000000,...,0.522988,0.563218,0.402299,inf,inf,inf,inf,NaN,0.398649,0.071131
25%,0.760638,0.765773,0.070011,0.155934,0.181395,0.111628,0.074384,0.052350,0.024386,0.000000,...,1.634843,1.187901,inf,inf,NaN,inf,NaN,NaN,0.434224,0.079517
50%,0.787234,0.801386,0.292484,0.312927,0.234700,0.164729,0.098236,0.069767,0.030835,0.011628,...,2.129339,inf,inf,inf,NaN,inf,NaN,NaN,0.457823,0.117473
75%,0.797872,0.832317,0.407383,0.386476,0.277715,0.200464,0.130967,0.092317,0.063352,0.031138,...,inf,inf,inf,inf,NaN,inf,NaN,NaN,0.501692,0.140719
max,0.840426,0.861258,0.552632,0.586207,0.482759,0.241379,0.205882,0.147059,0.147059,0.058824,...,inf,inf,inf,inf,inf,inf,inf,NaN,0.541101,0.256421


In [8]:
results_5 = pd.DataFrame([go(config,floor=True) for _ in range(10)])
results_5.describe()

,accuracy,balanced_acc,deo_0.30,deo_0.34,deo_0.38,deo_0.42,deo_0.46,deo_0.50,deo_0.54,deo_0.58,...,di_0.42,di_0.46,di_0.50,di_0.54,di_0.58,di_0.62,di_0.66,di_0.70,loss,renyi
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,9.000000,4.000000,1.000000,0.0,10.000000,10.000000
mean,0.789894,0.810831,0.335834,0.290261,0.240435,0.181334,0.156258,0.100428,0.066313,0.045396,...,inf,inf,inf,inf,inf,inf,inf,NaN,0.464705,0.180450
std,0.027950,0.057982,0.170592,0.161973,0.123629,0.095067,0.068256,0.053883,0.033652,0.032431,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.045063,0.086587
min,0.728723,0.669942,0.115385,0.045455,0.068627,0.098039,0.058824,0.029412,0.000000,0.000000,...,0.627273,0.511111,0.724138,0.402299,0.241379,inf,inf,NaN,0.400907,0.076497
25%,0.781915,0.809187,0.178571,0.153105,0.186086,0.120287,0.121130,0.061975,0.050420,0.024436,...,1.098900,0.957288,inf,inf,NaN,inf,NaN,NaN,0.446544,0.102497
50%,0.789894,0.820315,0.333333,0.315611,0.235294,0.177124,0.149206,0.088562,0.072807,0.029412,...,1.790909,inf,inf,inf,NaN,inf,NaN,NaN,0.463472,0.172053
75%,0.809840,0.851814,0.480556,0.364551,0.298810,0.188910,0.183824,0.133459,0.088235,0.079437,...,inf,inf,inf,inf,NaN,inf,NaN,NaN,0.482276,0.230720
max,0.824468,0.864217,0.558824,0.571429,0.485714,0.428571,0.287879,0.186275,0.114286,0.088235,...,inf,inf,inf,inf,inf,inf,inf,NaN,0.564968,0.338216


In [9]:
results_6 = pd.DataFrame([go(config) for _ in range(10)])
results_6.describe()

,accuracy,balanced_acc,deo_0.30,deo_0.34,deo_0.38,deo_0.42,deo_0.46,deo_0.50,deo_0.54,deo_0.58,...,di_0.42,di_0.46,di_0.50,di_0.54,di_0.58,di_0.62,di_0.66,di_0.70,loss,renyi
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.0,...,5.000000,5.000000,2.000000,1.000000,0.0,0.0,0.0,0.0,10.000000,10.000000
mean,0.796809,0.815869,0.119361,0.066911,0.062099,0.046783,0.033729,0.003571,0.003571,0.0,...,inf,inf,inf,inf,NaN,NaN,NaN,NaN,0.495094,0.100895
std,0.042612,0.060428,0.115507,0.074747,0.080223,0.077462,0.077294,0.011294,0.011294,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.054652,0.023780
min,0.739362,0.726614,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.105882,0.000000,inf,inf,NaN,NaN,NaN,NaN,0.416432,0.057522
25%,0.756649,0.771178,0.037226,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,inf,NaN,NaN,NaN,NaN,NaN,0.447822,0.091259
50%,0.811170,0.820186,0.081763,0.049156,0.038518,0.010638,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,inf,NaN,NaN,NaN,NaN,NaN,0.500838,0.098109
75%,0.819149,0.867324,0.171947,0.104551,0.069700,0.063539,0.028046,0.000000,0.000000,0.0,...,NaN,NaN,inf,NaN,NaN,NaN,NaN,NaN,0.545658,0.113638
max,0.867021,0.897303,0.321429,0.212121,0.226190,0.250000,0.250000,0.035714,0.035714,0.0,...,inf,inf,inf,inf,NaN,NaN,NaN,NaN,0.563212,0.144850


In [10]:
import sys
sys.path.append('../baselines/')
from ferm import go_ferm
from collections import namedtuple
encoded_data, to_protect, encoded_data_test, to_protect_test = read_drug()
encoded_data.drop(encoded_data.columns[0], axis=1, inplace=True)
encoded_data_test.drop(encoded_data_test.columns[0], axis=1, inplace=True)
encoded_data[encoded_data.columns[0]] = encoded_data[encoded_data.columns[0]].astype('category')
encoded_data_test[encoded_data_test.columns[0]] = encoded_data_test[encoded_data_test.columns[0]].astype('category')
train = namedtuple('_', 'data, target')(encoded_data.drop('Target',axis=1).values, encoded_data.Target.values*2-1)
test = namedtuple('_', 'data, target')(encoded_data_test.drop('Target',axis=1).values, encoded_data_test.Target.values*2-1)
go_ferm(train, test, sensible_feature=0)

Different values of the sensible feature 0 : [0, 1]
Grid search for SVM...
Best Estimator: SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Accuracy test: 0.7872340425531915
Accuracy train: 0.7972893341190336
Renyi test nan
Renyi train 0.009999999776482582
DEO test: 0.0
DEO train: 0.013357778941854373



Grid search for our method...
Best Fair Estimator: FERM(C=0.1, gamma=0.1, kernel='rbf', sensible_feature=0)
Accuracy test: 0.7872340425531915
Accuracy train: 0.7908073070123748
Renyi test 0.10619920492172241
Renyi train 0.105455182492733
DEO test: 0.0
DEO train: 0.0


(0.7872340425531915, 0.0)

In [12]:
for x in [results_1, results_2, results_3, results_4, results_5, results_6]:
    dx = x.describe()
    print( "$%.2f {\pm} %.2f $ & $ %.2f {\pm} %.2f$ & " % (dx['accuracy']['mean'], dx['accuracy']['std'],
                                                   dx['deo_0.50']['mean'], dx['deo_0.50']['std'])
             )

$0.79 {\pm} 0.03 $ & $ 0.15 {\pm} 0.16$ & 
$0.78 {\pm} 0.02 $ & $ 0.09 {\pm} 0.06$ & 
$0.78 {\pm} 0.05 $ & $ 0.00 {\pm} 0.00$ & 
$0.78 {\pm} 0.04 $ & $ 0.07 {\pm} 0.04$ & 
$0.79 {\pm} 0.03 $ & $ 0.10 {\pm} 0.05$ & 
$0.80 {\pm} 0.04 $ & $ 0.00 {\pm} 0.01$ & 


NameError: name 'to_protect' is not defined